# Face Recognition Model Training## Using UWA HSFD Hyperspectral DatabaseThis notebook demonstrates building a face recognition model using the UWA HSFD hyperspectral database. We preprocess hyperspectral data to grayscale for simplicity and train a CNN to output face embeddings.

In [ ]:
import torchimport torch.nn as nnimport torch.optim as optimfrom torchvision import models, transformsimport numpy as npimport matplotlib.pyplot as pltimport cv2from sklearn.model_selection import train_test_splitimport os

## 1. Data PreprocessingThe UWA HSFD database contains hyperspectral images. We'll convert them to grayscale for simplicity.**Note**: The UWA HSFD database is located at the specified path and contains PNG files.Dataset structure expected:```UWA_HSFD/  ├── subject_01/  │   ├── image_001.png  │   └── ...  ├── subject_02/  └── ...```

In [ ]:
# Set the data pathdata_path = r"C:\Users\Anvitha\Face based Person Authentication\UWA HSFD V1.1 (1)\UWA HSFD V1.1\HyperSpec_Face_Session1"print(f"Data path set to: {data_path}")

In [ ]:
def load_and_preprocess_data(data_dir, target_size=(224, 224)):
    """
    Load and preprocess UWA HSFD data from PNG files.
    
    Args:
        data_dir: Path to data directory
        target_size: Target image size
        
    Returns:
        X: Image data array (RGB)
        y: Labels array
        labels: Label mapping
    """
    images = []
    labels = []
    label_map = {}
    current_label = 0
    
    if os.path.exists(data_dir):
        for subject_dir in sorted(os.listdir(data_dir)):
            subject_path = os.path.join(data_dir, subject_dir)
            if os.path.isdir(subject_path):
                label_map[subject_dir] = current_label
                
                for image_file in os.listdir(subject_path):
                    if image_file.endswith('.png'):
                        image_path = os.path.join(subject_path, image_file)
                        
                        # Load PNG image
                        img = cv2.imread(image_path)
                        if img is not None:
                            # Convert grayscale to RGB if necessary
                            if len(img.shape) == 2:
                                img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
                            
                            # Resize to target size
                            img = cv2.resize(img, target_size)
                            
                            # Normalize to 0-1
                            img = img.astype(np.float32) / 255.0
                            
                            images.append(img)
                            labels.append(current_label)
                        
                current_label += 1
    
    return np.array(images), np.array(labels), label_map

## 2. Load Data

In [ ]:
# Load the dataX, y, label_map = load_and_preprocess_data(data_path)print(f"Loaded {len(X)} images from {len(label_map)} subjects")print(f"Image shape: {X.shape[1:]}")print(f"Label map: {label_map}")

## 3. Build CNN Model for Face EmbeddingsWe'll create a CNN model that outputs face embeddings. We'll use transfer learning with MobileNetV2 as the base.

In [ ]:
def build_face_embedding_model(embedding_dim=1280):
    """
    Build a CNN model for face embedding extraction using MobileNetV2.
    
    Args:
        embedding_dim: Dimension of output embedding (1280 for MobileNetV2)
        
    Returns:
        PyTorch Model
    """
    # Load pre-trained MobileNetV2
    model = models.mobilenet_v2(pretrained=True)
    
    # Remove the classifier (last layer)
    model.classifier = nn.Sequential(*list(model.classifier.children())[:-1])
    
    # Add a linear layer to get the desired embedding dimension
    model.embedding = nn.Linear(1280, embedding_dim)
    
    return model

In [ ]:
# Create the modelmodel = build_face_embedding_model(embedding_dim=1280)device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')model = model.to(device)model.eval()  # Set to evaluation modeprint(f"Model created and moved to {device}")

## 4. Triplet Loss FunctionFor face recognition, we use triplet loss to train the network to produce similar embeddings for the same person and different embeddings for different people.

In [ ]:
def triplet_loss(anchor, positive, negative, margin=0.2):
    """
    Triplet loss function.
    
    Args:
        anchor: Anchor embeddings
        positive: Positive embeddings
        negative: Negative embeddings
        margin: Margin parameter
        
    Returns:
        Loss value
    """
    pos_dist = torch.sum((anchor - positive) ** 2, dim=1)
    neg_dist = torch.sum((anchor - negative) ** 2, dim=1)
    loss = torch.mean(torch.relu(pos_dist - neg_dist + margin))
    return loss

## 5. Training (Demonstration)This section demonstrates the training process. In practice, you would:1. Load the actual UWA HSFD dataset2. Preprocess hyperspectral images to grayscale/RGB3. Create triplet batches (anchor, positive, negative)4. Train the model using triplet loss

In [ ]:
# Split data for training/validationX_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)# For demonstration, compile and train (would need triplet data generator)# optimizer = optim.Adam(model.parameters(), lr=0.001)# model.train()# # Training loop here...print("Data loaded and ready for training.")print(f"Training samples: {len(X_train)}")print(f"Validation samples: {len(X_val)}")print("For actual training, implement triplet data generator.")

## 6. Using Pre-trained ModelFor the face authentication system, we use a pre-trained MobileNetV2 model which is already integrated in the `face_utils.py` module. This provides a practical solution that works with RGB webcam inputs without requiring extensive training on hyperspectral data.

In [ ]:
# Load the embedding extractor used in the systemimport syssys.path.append('..')from src.face_utils import EmbeddingExtractorextractor = EmbeddingExtractor()print("Embedding extractor ready!")print(f"Input shape: {extractor.input_shape}")print(f"Model architecture:")print(extractor.model)

## 7. Testing Embedding ExtractionLet's test the embedding extraction with a sample image.

In [ ]:
# Create a test image (random for demonstration)test_image = np.random.randint(0, 255, (224, 224, 3), dtype=np.uint8)# Extract embeddingembedding = extractor.extract_embedding(test_image)normalized_embedding = extractor.normalize_embedding(embedding)print(f"Embedding shape: {embedding.shape}")print(f"Embedding dimension: {len(embedding)}")print(f"Embedding L2 norm (before normalization): {np.linalg.norm(embedding):.4f}")print(f"Embedding L2 norm (after normalization): {np.linalg.norm(normalized_embedding):.4f}")

## 8. Similarity ComparisonDemonstrate how embeddings are compared using cosine similarity.

In [ ]:
from src.auth_utils import cosine_similarity# Create two test embeddingsembedding1 = np.random.randn(1280)embedding2 = np.random.randn(1280)embedding3 = embedding1 + np.random.randn(1280) * 0.1  # Similar to embedding1# Calculate similaritiessim_different = cosine_similarity(embedding1, embedding2)sim_similar = cosine_similarity(embedding1, embedding3)sim_same = cosine_similarity(embedding1, embedding1)print(f"Similarity (different faces): {sim_different:.4f}")print(f"Similarity (similar faces): {sim_similar:.4f}")print(f"Similarity (same face): {sim_same:.4f}")print("\nTypical threshold for authentication: 0.6")

## SummaryThis notebook demonstrates:1. How to load and preprocess UWA HSFD PNG images (handling grayscale)2. Building a CNN for face embedding extraction3. Triplet loss for training face recognition models4. Using pre-trained models for practical applications5. Embedding extraction and similarity comparisonThe actual face authentication system uses MobileNetV2 pre-trained on ImageNet, which provides excellent results for RGB webcam inputs without requiring extensive training on hyperspectral data.